# 📊 Análise de Pré-processamento de Dados

Este notebook analisa um dataset e determina automaticamente se você deve usar:
- ✅ Apenas escalonamento
- ✅ Apenas transformação
- ✅ Transformação + escalonamento
- ✅ Nenhum dos dois

---

## 📦 Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, normaltest, skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

# Configuração de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Bibliotecas importadas com sucesso!")

## 📂 Carregamento dos Dados

**Instruções:** Altere o caminho do arquivo para o seu dataset.

In [ ]:
# Carregue seu dataset aqui
df = pd.read_csv('/mnt/user-data/uploads/1761482687687_pasted-content-1761482687686.txt')

print(f"✓ Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
print(f"\nPrimeiras linhas:")
df.head()

## 🔧 Classe Analisadora

In [ ]:
class AnalisadorPreProcessamento:
    """Classe para analisar necessidade de pré-processamento"""
    
    def __init__(self, df, target_col='target'):
        """
        Inicializa o analisador
        
        Parameters:
        -----------
        df : pd.DataFrame
            Dataset a ser analisado
        target_col : str
            Nome da coluna target (será excluída da análise)
        """
        self.df = df.copy()
        self.target_col = target_col
        
        # Separar features numéricas (excluindo target e colunas não numéricas)
        self.features = self.df.select_dtypes(include=[np.number]).columns.tolist()
        if target_col in self.features:
            self.features.remove(target_col)
        
        # Remover colunas com todos valores iguais ou NaN
        self.features = [col for col in self.features 
                        if self.df[col].nunique() > 1 and self.df[col].notna().sum() > 0]
        
        self.resultados = {}
        
    def calcular_estatisticas(self):
        """Calcula estatísticas descritivas de cada feature"""
        print("=" * 80)
        print("📊 ESTATÍSTICAS DESCRITIVAS")
        print("=" * 80)
        
        estatisticas = []
        
        for col in self.features:
            dados = self.df[col].dropna()
            
            if len(dados) == 0:
                continue
                
            stats_dict = {
                'Feature': col,
                'Min': dados.min(),
                'Max': dados.max(),
                'Mean': dados.mean(),
                'Median': dados.median(),
                'Std': dados.std(),
                'Range': dados.max() - dados.min(),
                'Zeros_%': (dados == 0).sum() / len(dados) * 100,
                'Skewness': skew(dados),
                'Kurtosis': kurtosis(dados)
            }
            estatisticas.append(stats_dict)
        
        self.stats_df = pd.DataFrame(estatisticas)
        print(f"\n✓ Analisadas {len(self.features)} features numéricas\n")
        
        return self.stats_df
    
    def analisar_escala(self):
        """Analisa a necessidade de escalonamento"""
        print("\n" + "=" * 80)
        print("📏 ANÁLISE DE ESCALA")
        print("=" * 80)
        
        # Calcular ranges e magnitudes
        ranges = self.stats_df['Range'].values
        max_range = ranges.max()
        min_range = ranges[ranges > 0].min() if any(ranges > 0) else 0
        
        # Verificar disparidade de escalas
        if min_range > 0:
            ratio_escala = max_range / min_range
        else:
            ratio_escala = np.inf
        
        print(f"\n📊 Estatísticas de Escala:")
        print(f"   • Range mínimo: {min_range:.2e}")
        print(f"   • Range máximo: {max_range:.2e}")
        print(f"   • Razão max/min: {ratio_escala:.2e}")
        
        # Critérios para recomendar escalonamento
        precisa_escalonamento = False
        razoes = []
        
        if ratio_escala > 100:
            precisa_escalonamento = True
            razoes.append(f"Grande disparidade de escalas (razão = {ratio_escala:.0f}x)")
        
        # Verificar features com diferentes ordens de magnitude
        magnitudes = self.stats_df['Max'].apply(lambda x: np.floor(np.log10(abs(x) + 1)))
        diff_magnitudes = magnitudes.max() - magnitudes.min()
        
        if diff_magnitudes > 3:
            precisa_escalonamento = True
            razoes.append(f"Features com diferentes ordens de magnitude (diferença: {diff_magnitudes:.0f})")
        
        self.resultados['escalonamento'] = {
            'necessario': precisa_escalonamento,
            'razoes': razoes,
            'ratio_escala': ratio_escala,
            'diff_magnitudes': diff_magnitudes
        }
        
        if precisa_escalonamento:
            print(f"\n✅ ESCALONAMENTO RECOMENDADO")
            print(f"\nMotivos:")
            for r in razoes:
                print(f"   • {r}")
        else:
            print(f"\n❌ ESCALONAMENTO NÃO NECESSÁRIO")
            print(f"   • Escalas relativamente uniformes")
        
        return precisa_escalonamento
    
    def analisar_normalidade(self):
        """Analisa a normalidade e necessidade de transformação"""
        print("\n" + "=" * 80)
        print("📈 ANÁLISE DE NORMALIDADE E ASSIMETRIA")
        print("=" * 80)
        
        features_problematicas = []
        
        for col in self.features[:20]:  # Análise detalhada das primeiras 20
            dados = self.df[col].dropna()
            
            if len(dados) < 3:
                continue
            
            # Calcular métricas
            sk = skew(dados)
            kt = kurtosis(dados)
            
            # Teste de normalidade
            if len(dados) >= 8:
                try:
                    _, p_shapiro = shapiro(dados[:5000])  # Limitar para performance
                except:
                    p_shapiro = 1.0
            else:
                p_shapiro = 1.0
            
            # Identificar problemas
            problemas = []
            if abs(sk) > 1:
                problemas.append(f"Alta assimetria ({sk:.2f})")
            if abs(kt) > 3:
                problemas.append(f"Curtose anormal ({kt:.2f})")
            if p_shapiro < 0.05:
                problemas.append("Não-normal (p<0.05)")
            
            if problemas:
                features_problematicas.append({
                    'feature': col,
                    'skewness': sk,
                    'kurtosis': kt,
                    'p_value': p_shapiro,
                    'problemas': problemas
                })
        
        print(f"\n📊 Resumo da Análise:")
        print(f"   • Features analisadas: {min(20, len(self.features))}")
        print(f"   • Features com problemas: {len(features_problematicas)}")
        
        if features_problematicas:
            print(f"\n⚠️  Features mais problemáticas (top 10):")
            sorted_features = sorted(features_problematicas, 
                                   key=lambda x: abs(x['skewness']), 
                                   reverse=True)[:10]
            
            for i, feat in enumerate(sorted_features, 1):
                print(f"\n   {i}. {feat['feature']}")
                print(f"      Skewness: {feat['skewness']:.3f}")
                print(f"      Kurtosis: {feat['kurtosis']:.3f}")
                print(f"      p-value: {feat['p_value']:.4f}")
        
        # Análise de assimetria geral
        skewness_values = self.stats_df['Skewness'].abs()
        
        pct_alta_assimetria = (skewness_values > 1).sum() / len(skewness_values) * 100
        pct_moderada_assimetria = ((skewness_values > 0.5) & (skewness_values <= 1)).sum() / len(skewness_values) * 100
        
        print(f"\n📈 Distribuição de Assimetria:")
        print(f"   • Alta assimetria (|skew| > 1): {pct_alta_assimetria:.1f}%")
        print(f"   • Moderada (0.5 < |skew| ≤ 1): {pct_moderada_assimetria:.1f}%")
        print(f"   • Baixa (|skew| ≤ 0.5): {100 - pct_alta_assimetria - pct_moderada_assimetria:.1f}%")
        
        # Decisão sobre transformação
        precisa_transformacao = pct_alta_assimetria > 30 or len(features_problematicas) > len(self.features) * 0.3
        
        self.resultados['transformacao'] = {
            'necessario': precisa_transformacao,
            'pct_alta_assimetria': pct_alta_assimetria,
            'features_problematicas': len(features_problematicas),
            'total_features': len(self.features)
        }
        
        if precisa_transformacao:
            print(f"\n✅ TRANSFORMAÇÃO RECOMENDADA")
            print(f"\nMotivos:")
            print(f"   • {pct_alta_assimetria:.1f}% das features têm alta assimetria")
            print(f"   • {len(features_problematicas)} features com distribuição problemática")
        else:
            print(f"\n❌ TRANSFORMAÇÃO NÃO NECESSÁRIA")
            print(f"   • Maioria das features tem distribuição aceitável")
        
        return precisa_transformacao
    
    def analisar_outliers(self):
        """Analisa a presença de outliers"""
        print("\n" + "=" * 80)
        print("🎯 ANÁLISE DE OUTLIERS")
        print("=" * 80)
        
        outliers_info = []
        
        for col in self.features:
            dados = self.df[col].dropna()
            
            if len(dados) < 4:
                continue
            
            Q1 = dados.quantile(0.25)
            Q3 = dados.quantile(0.75)
            IQR = Q3 - Q1
            
            lower_bound = Q1 - 3 * IQR
            upper_bound = Q3 + 3 * IQR
            
            outliers = ((dados < lower_bound) | (dados > upper_bound)).sum()
            pct_outliers = outliers / len(dados) * 100
            
            if pct_outliers > 0:
                outliers_info.append({
                    'feature': col,
                    'n_outliers': outliers,
                    'pct_outliers': pct_outliers
                })
        
        # Ordenar por percentual
        outliers_info.sort(key=lambda x: x['pct_outliers'], reverse=True)
        
        print(f"\n📊 Resumo:")
        print(f"   • Features com outliers: {len(outliers_info)}/{len(self.features)}")
        
        if outliers_info:
            print(f"\n⚠️  Top 10 features com mais outliers:")
            for i, info in enumerate(outliers_info[:10], 1):
                print(f"   {i}. {info['feature']}: {info['n_outliers']} outliers ({info['pct_outliers']:.2f}%)")
        
        # Calcular média de outliers
        if outliers_info:
            media_pct_outliers = np.mean([x['pct_outliers'] for x in outliers_info])
            print(f"\n   Média de outliers: {media_pct_outliers:.2f}%")
            
            if media_pct_outliers > 5:
                print(f"\n💡 DICA: Transformações como Yeo-Johnson podem ajudar a reduzir o impacto dos outliers")
        
        self.resultados['outliers'] = outliers_info
        
        return outliers_info
    
    def gerar_visualizacoes(self, n_features=6):
        """Gera visualizações das features mais problemáticas"""
        print("\n" + "=" * 80)
        print("📊 GERANDO VISUALIZAÇÕES")
        print("=" * 80)
        
        # Selecionar features com maior assimetria
        features_sorted = self.stats_df.nlargest(n_features, 'Skewness')['Feature'].tolist()
        
        fig, axes = plt.subplots(n_features, 2, figsize=(15, 4*n_features))
        
        for i, col in enumerate(features_sorted):
            dados = self.df[col].dropna()
            
            # Histograma
            axes[i, 0].hist(dados, bins=50, edgecolor='black', alpha=0.7)
            axes[i, 0].set_title(f'{col}\nHistograma', fontweight='bold')
            axes[i, 0].set_xlabel('Valor')
            axes[i, 0].set_ylabel('Frequência')
            
            # Adicionar estatísticas
            sk = skew(dados)
            kt = kurtosis(dados)
            axes[i, 0].text(0.02, 0.98, f'Skew: {sk:.2f}\nKurt: {kt:.2f}',
                          transform=axes[i, 0].transAxes,
                          verticalalignment='top',
                          bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
            
            # Boxplot
            axes[i, 1].boxplot(dados, vert=False)
            axes[i, 1].set_title(f'{col}\nBoxplot', fontweight='bold')
            axes[i, 1].set_xlabel('Valor')
        
        plt.tight_layout()
        plt.show()
        print("✓ Visualizações de distribuições geradas")
        
        # Gráfico de resumo de assimetria
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Selecionar top 20 features por assimetria
        top_skewed = self.stats_df.nlargest(20, 'Skewness', keep='all')
        
        colors = ['red' if abs(x) > 1 else 'orange' if abs(x) > 0.5 else 'green' 
                 for x in top_skewed['Skewness']]
        
        ax.barh(range(len(top_skewed)), top_skewed['Skewness'], color=colors, alpha=0.7)
        ax.set_yticks(range(len(top_skewed)))
        ax.set_yticklabels(top_skewed['Feature'], fontsize=8)
        ax.set_xlabel('Skewness', fontweight='bold')
        ax.set_title('Top 20 Features por Assimetria', fontsize=14, fontweight='bold')
        ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
        ax.axvline(x=-1, color='red', linestyle=':', alpha=0.5, label='|skew| = 1')
        ax.axvline(x=1, color='red', linestyle=':', alpha=0.5)
        ax.grid(axis='x', alpha=0.3)
        ax.legend()
        
        plt.tight_layout()
        plt.show()
        print("✓ Gráfico de assimetria gerado")
    
    def gerar_recomendacao_final(self):
        """Gera recomendação final com base em todas as análises"""
        print("\n" + "=" * 80)
        print("🎯 RECOMENDAÇÃO FINAL")
        print("=" * 80)
        
        precisa_escalonamento = self.resultados['escalonamento']['necessario']
        precisa_transformacao = self.resultados['transformacao']['necessario']
        
        print(f"\n📋 Análise Completa:")
        print(f"   • Escalonamento necessário: {'SIM ✅' if precisa_escalonamento else 'NÃO ❌'}")
        print(f"   • Transformação necessária: {'SIM ✅' if precisa_transformacao else 'NÃO ❌'}")
        
        print(f"\n" + "─" * 80)
        
        if precisa_transformacao and precisa_escalonamento:
            print(f"\n🔧 RECOMENDAÇÃO: TRANSFORMAÇÃO + ESCALONAMENTO")
            print(f"\n📝 Pipeline Recomendado:")
            print(f"   1️⃣  Yeo-Johnson ou Box-Cox (transformação)")
            print(f"   2️⃣  Seleção de Features")
            print(f"   3️⃣  StandardScaler ou RobustScaler (escalonamento)")
            print(f"   4️⃣  Modelo")
            
            print(f"\n💡 Justificativa:")
            print(f"   • Dados apresentam alta assimetria ({self.resultados['transformacao']['pct_alta_assimetria']:.1f}% features)")
            print(f"   • Escalas muito diferentes entre features (razão: {self.resultados['escalonamento']['ratio_escala']:.0f}x)")
            print(f"   • Transformação corrige distribuições antes da seleção de features")
            print(f"   • Escalonamento garante magnitude comparável para o modelo")
            
        elif precisa_transformacao:
            print(f"\n🔧 RECOMENDAÇÃO: APENAS TRANSFORMAÇÃO")
            print(f"\n📝 Pipeline Recomendado:")
            print(f"   1️⃣  Yeo-Johnson ou Box-Cox (transformação)")
            print(f"   2️⃣  Seleção de Features")
            print(f"   3️⃣  Modelo")
            
            print(f"\n💡 Justificativa:")
            print(f"   • Dados apresentam alta assimetria ({self.resultados['transformacao']['pct_alta_assimetria']:.1f}% features)")
            print(f"   • Escalas relativamente uniformes")
            print(f"   • Transformação melhorará a normalidade para modelos que assumem isso")
            
        elif precisa_escalonamento:
            print(f"\n🔧 RECOMENDAÇÃO: APENAS ESCALONAMENTO")
            print(f"\n📝 Pipeline Recomendado:")
            print(f"   1️⃣  Seleção de Features (se usar métodos sensíveis à escala)")
            print(f"   2️⃣  StandardScaler ou MinMaxScaler")
            print(f"   3️⃣  Modelo")
            
            print(f"\n💡 Justificativa:")
            print(f"   • Grande disparidade de escalas (razão: {self.resultados['escalonamento']['ratio_escala']:.0f}x)")
            print(f"   • Distribuições relativamente normais")
            print(f"   • Escalonamento evitará dominância de features com valores grandes")
            
        else:
            print(f"\n🔧 RECOMENDAÇÃO: NENHUM PRÉ-PROCESSAMENTO OBRIGATÓRIO")
            print(f"\n📝 Pipeline Recomendado:")
            print(f"   1️⃣  Seleção de Features (métodos baseados em árvores funcionam bem)")
            print(f"   2️⃣  Modelo")
            
            print(f"\n💡 Justificativa:")
            print(f"   • Escalas relativamente uniformes")
            print(f"   • Distribuições aceitáveis")
            print(f"   • Considere usar modelos baseados em árvores (Random Forest, XGBoost)")
        
        print(f"\n" + "─" * 80)
        
        return precisa_transformacao, precisa_escalonamento

print("✅ Classe AnalisadorPreProcessamento criada!")

## 🚀 Execução da Análise

### Configuração

**Importante:** Altere o nome da coluna target se necessário.

In [ ]:
# Criar instância do analisador
analisador = AnalisadorPreProcessamento(df, target_col='target')

print(f"✓ Analisador criado com sucesso!")
print(f"✓ Features numéricas identificadas: {len(analisador.features)}")

### 1️⃣ Estatísticas Descritivas

In [ ]:
stats_df = analisador.calcular_estatisticas()

# Visualizar as 10 primeiras features
stats_df.head(10)

### 2️⃣ Análise de Escala

In [ ]:
precisa_escalonamento = analisador.analisar_escala()

### 3️⃣ Análise de Normalidade e Assimetria

In [ ]:
precisa_transformacao = analisador.analisar_normalidade()

### 4️⃣ Análise de Outliers

In [ ]:
outliers_info = analisador.analisar_outliers()

### 5️⃣ Visualizações

In [ ]:
# Gerar visualizações das 6 features mais assimétricas
analisador.gerar_visualizacoes(n_features=6)

### 6️⃣ Recomendação Final

In [ ]:
precisa_transformacao, precisa_escalonamento = analisador.gerar_recomendacao_final()

## 💻 Exemplo de Pipeline

Com base na análise acima, aqui está um exemplo de código para implementar o pipeline recomendado:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer, StandardScaler, RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

# Separar features e target
X = df.drop(columns=['target', 'os_timestamp', 'node_name'], errors='ignore')
y = df['target']

# Manter apenas features numéricas
X = X.select_dtypes(include=[np.number])

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✓ Dados preparados:")
print(f"   • X_train: {X_train.shape}")
print(f"   • X_test: {X_test.shape}")
print(f"   • Features: {X.shape[1]}")

In [ ]:
# Pipeline baseado na recomendação
if precisa_transformacao and precisa_escalonamento:
    print("📝 Criando pipeline com: Transformação + Seleção + Escalonamento + Modelo")
    pipeline = Pipeline([
        ('transformer', PowerTransformer(method='yeo-johnson')),
        ('selector', SelectKBest(f_classif, k=20)),
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(max_iter=1000))
    ])
    
elif precisa_transformacao:
    print("📝 Criando pipeline com: Transformação + Seleção + Modelo")
    pipeline = Pipeline([
        ('transformer', PowerTransformer(method='yeo-johnson')),
        ('selector', SelectKBest(f_classif, k=20)),
        ('model', LogisticRegression(max_iter=1000))
    ])
    
elif precisa_escalonamento:
    print("📝 Criando pipeline com: Seleção + Escalonamento + Modelo")
    pipeline = Pipeline([
        ('selector', SelectKBest(f_classif, k=20)),
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(max_iter=1000))
    ])
    
else:
    print("📝 Criando pipeline com: Seleção + Modelo (árvores)")
    from sklearn.ensemble import RandomForestClassifier
    pipeline = Pipeline([
        ('selector', SelectKBest(f_classif, k=20)),
        ('model', RandomForestClassifier(n_estimators=100, random_state=42))
    ])

print("✅ Pipeline criado!")

In [ ]:
# Treinar e avaliar
print("🚀 Treinando o modelo...\n")

# Cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')

print(f"📊 Resultados da Validação Cruzada (5-fold):")
print(f"   • Acurácias: {scores}")
print(f"   • Média: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

# Treinar no conjunto completo de treino
pipeline.fit(X_train, y_train)

# Avaliar no teste
test_score = pipeline.score(X_test, y_test)
print(f"\n✓ Acurácia no conjunto de teste: {test_score:.4f}")

## 📊 Exportar Estatísticas

In [ ]:
# Salvar estatísticas em CSV
stats_df.to_csv('estatisticas_features.csv', index=False)
print("✅ Estatísticas salvas em: estatisticas_features.csv")

# Visualizar resumo
print("\n📋 Resumo das Estatísticas:")
print(f"   • Total de features analisadas: {len(stats_df)}")
print(f"   • Features com alta assimetria (|skew|>1): {(stats_df['Skewness'].abs() > 1).sum()}")
print(f"   • Range médio: {stats_df['Range'].mean():.2e}")
print(f"   • Desvio padrão médio: {stats_df['Std'].mean():.2e}")

## 📝 Conclusões

### Resumo da Análise:

Execute todas as células acima para ver:
- ✅ Se seu dataset precisa de **transformação** (Yeo-Johnson/Box-Cox)
- ✅ Se seu dataset precisa de **escalonamento** (StandardScaler/MinMaxScaler)
- ✅ A **ordem correta** do pipeline de pré-processamento
- ✅ Visualizações das distribuições mais problemáticas
- ✅ Código pronto para implementação

### 💡 Dicas Finais:

1. **Sempre use Pipelines** do scikit-learn para evitar data leakage
2. **Transformação vem antes** da seleção de features
3. **Escalonamento vem depois** da seleção de features
4. Para **modelos baseados em árvores**, transformação e escalonamento geralmente não são necessários
5. Use **PowerTransformer** com `method='yeo-johnson'` pois aceita valores negativos

---

**🎉 Notebook criado com sucesso!**